In [ ]:
__author__ = "Tim Cuddeback"

import yfpy, json, ast
from itertools import chain
from yfpy.query import YahooFantasySportsQuery
from yfpy.utils import unpack_data
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
# Function that pulls data from Yahoo's Fantasy API and returns a sorted list of tuples with (team_id,current points)
league_id = '12514' # put your real league id here
game_id = "399" # put the game id here (game id's reflect the type of sport and the year)
game_code = "nfl" # put the game code here
season = "2020" # put the year of the current fantasy season
auth_dir = '' # put the location where you are storing the client_id/secret
working_dir = '' # put the location where you want excel file to be saved
excel_file_name = 'Points.xlsx' # Name your file after you set up the headers

# Query yffpy to get data
yahoo_query = YahooFantasySportsQuery(auth_dir, league_id, game_id=game_id, game_code=game_code, offline=False)

# Get standings data to get the points for the day
data = yahoo_query.get_team_roster_by_week(1,1)
# .get_league_teams(self)
# .get_league_players(self)
# .get_league_draft_results(self)
# .get_league_matchups_by_week(self,chosen_week)
# .get_team_info(self,team_id)
# .get_team_standings(self,team_id) # has current "streak" within
# .get_player_stats_by_week(self, player_key, chosen_week="current")